In [1]:
import json
import pandas as pd
import numpy as np
import os
import time
import requests

logic: 
1. first get the max number of external slots: 200
2. then filter all the validators who are eligible to be selected next epoch; if the total number of validators is lower than 200, then every filtered validator should be elected next epoch; if total number is higher than 200, we need to first sort these validators based on their total delegations, and get the top 200 validators.
3. wait for one epoch, to check whether those filtered validators are elected

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getCommittees():
    method = "hmy_getSuperCommittees"
    params = []
    return get_information(method, params)['result']['current']

In [4]:
def getAllValidator():
    method = 'hmy_getAllValidatorAddresses'
    params = []
    return get_information(method, params)['result'] 

In [5]:
def getValidatorInfo(validator):
    method = "hmy_getValidatorInformation"
    params = [validator]
    return get_information(method, params)['result']

In [6]:
def getEpoch():
    method = "hmy_getEpoch"
    params = []
    epoch = get_information(method, params)['result']
    return int(epoch, 16)

In [7]:
def E1_test():
    flag = True
    committees = getCommittees()
    slot = committees['external-slot-count']
    validator = getAllValidator()
    epoch = getEpoch()
    
    # get top #slots nodes who are eligible to elected next epoch
    eligible = []
    stake = dict()
    for i in validator:
        info = getValidatorInfo(i)
        status = info['epos-status']
        if status == 'currently elected and signing enough blocks to be eligible for election next epoch' or \
        status == 'eligible to be elected next epoch':
            eligible.append(i)
            stake[i] = info['total-delegation']

    if len(eligible) > slot:
        sorted_stake = sorted(stake.items(), key=lambda kv: kv[1], reverse = True)
        eligible = [kv[0] for kv in sorted_stake[:slot]]
    
    # wait for epoch changes
    new_epoch = getEpoch()
    while new_epoch == epoch:
        time.sleep(8)
        new_epoch = getEpoch()
        
    # check whether the eligible validators are selected
    for i in eligible:
        info = getValidatorInfo(i)
        if info['epos-status'] == 'eligible to be elected next epoch' or \
        info['epos-status'] == 'not eligible to be elected next epoch':
            print("validator " + i + " who is eligible to be selected is not selected")
            flag = False
    if flag:
        return True
    return False

In [8]:
E1_test()

validator one164e2dwupqxd7ssr85ncnkx3quk7fexy0eta2vy who is eligible to be selected is not selected
validator one12mx7ssutgvd243g5vpvefl5sf6vttqkq5z3x04 who is eligible to be selected is not selected


False

the info of the exceptions: 

In [9]:
i = 'one164e2dwupqxd7ssr85ncnkx3quk7fexy0eta2vy'
info = getValidatorInfo(i)
info

{'validator': {'bls-public-keys': ['2eba33b81d0aacb22e793625077d0c57dd6d38d753bbf39dc720dea0f4a5c056c81d8967a99c2cf1811d0327b049ab16',
   '394d726edbb908fa71bdb2b417dd0fe7c77d9f86b3a0b1e1b51d11e26dd1cddb4e21269c3e1d08e75d43ec98ff4ca582',
   '73ac6379c785142d46bf8423469639fa1dfb9e416e97206cff2c07522a18999a54faaf9cbcad1e34a6c2dc770224848e',
   'b682337f407ec929e7cc18e5103da8a038d63118990b9d8124b31836d8b394a695f16d88b6a32b2543b1dcaaba99a582'],
  'last-epoch-in-committee': 0,
  'min-self-delegation': 322222000000000000000000,
  'max-total-delegation': 10000000000000000000000000,
  'rate': '0.030000000000000000',
  'max-rate': '0.700000000000000000',
  'max-change-rate': '0.030000000000000000',
  'update-height': 1014,
  'name': 'leo-s2',
  'identity': 'leo-sentry-s2',
  'website': 'https://sentry2.leochen.net',
  'security-contact': 'sentrys2@leochen.net',
  'details': 'sentry node of leo chen, shard2',
  'creation-height': 1014,
  'address': 'one164e2dwupqxd7ssr85ncnkx3quk7fexy0eta2vy',
 

In [10]:
i = 'one12mx7ssutgvd243g5vpvefl5sf6vttqkq5z3x04'
info = getValidatorInfo(i)
info

{'validator': {'bls-public-keys': ['4db5eeb4ff5379f7ba51d522fb17a07571cf427fae86cab31e9927e5048339d7a3c40941b1ce485f75f750aabbf69d82'],
  'last-epoch-in-committee': 0,
  'min-self-delegation': 10000000000000000000000,
  'max-total-delegation': 100000000000000000000000,
  'rate': '0.100000000000000000',
  'max-rate': '0.100000000000000000',
  'max-change-rate': '0.100000000000000000',
  'update-height': 25363,
  'name': 'Yishuang-2',
  'identity': 'Yishuang sentry',
  'website': 'yishuang@harmony.one',
  'security-contact': 'shuang',
  'details': 'Yishuang’s second validator',
  'creation-height': 25363,
  'address': 'one12mx7ssutgvd243g5vpvefl5sf6vttqkq5z3x04',
  'delegations': [{'delegator-address': 'one12mx7ssutgvd243g5vpvefl5sf6vttqkq5z3x04',
    'amount': 10000000000000000000000,
    'reward': 0,
    'undelegations': []}]},
 'current-epoch-performance': None,
 'metrics': None,
 'total-delegation': 10000000000000000000000,
 'currently-in-committee': False,
 'epos-status': 'eligible 